In [3]:
%reload_ext autoreload
%autoreload 2
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
from utils import *

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

Available devices:  2


In [2]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.1+cu121
cudnn version: 8902
cuda version: 12.1


In [4]:

max_length = 2048
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [8]:
print(tokens)

['.' '/' '0.3997395833333333' '0.4440104166666667' '0.5' '0.5703125'
 '0.6666666666666666' '0.75' '0.7994791666666666' '0.8880208333333334'
 '1.0' '1.1419270833333333' '1.3333333333333333' '1.5'
 '1.5989583333333333' '1.7135416666666667' '128 Feel' '2.0' '2.25'
 '2.3997395833333335' '2.6666666666666665' '3.0' '4.0' ':|' '<end>'
 '<pad>' '<start>' 'A' 'A major' 'A minor' 'A#' 'A##' 'Ab' 'Ab major'
 'Ab minor' 'Abb' 'Afoxé' 'Afro' 'B' 'B major' 'B minor' 'B#' 'B##'
 'Baião' 'Ballad' 'Bb' 'Bb major' 'Bb minor' 'Bbb' 'Blues' 'Bolero'
 'Bolero-Cha' 'Bossa' 'C' 'C major' 'C minor' 'C#' 'C##' 'Calypso' 'Cb'
 'Cbb' 'Cha Cha' 'Chacarera' 'Choro' 'Country Ballad' 'D' 'D major'
 'D minor' 'D#' 'D##' 'Db' 'Db major' 'Db minor' 'Dbb' 'Deliberately'
 'Disco' 'Dreamlike' 'E' 'E major' 'E minor' 'E#' 'E##' 'Eb' 'Eb major'
 'Eb minor' 'Ebb' 'Even 16ths' 'Even 8ths' 'F' 'F major' 'F minor' 'F#'
 'F##' 'Fb' 'Fbb' 'Folk' 'Form_A' 'Form_B' 'Form_C' 'Form_Coda' 'Form_D'
 'Form_Segno' 'Form_intro' 'Form_vers

In [5]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 2048) (4800, 2048) (43272, 2048, 8) (4800, 2048, 8)
data has 43272 pieces, 196 unique tokens.
data has 4800 pieces, 196 unique tokens.


In [6]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 250,
    }
)

03/12/2024 12:13:45 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [7]:
epochs = 250
embedding = 192
heads = 4
layers = 4
batch_size = 64
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

03/12/2024 12:13:48 - INFO - model -   number of parameters: 1.861760e+06
03/12/2024 12:13:48 - INFO - model -   number of parameters: 1.861760e+06


../models/model_epochs->250_heads->4_embd->192_batch->64_new_midi_embeddings
(2048, 8)(2048, 8)(2048, 8)

  0%|          | 0/677 [00:00<?, ?it/s]




(2048, 8)(2048, 8)(2048, 8)


(2048, 8)
(2048, 8)
(2048, 8)

  0%|          | 0/677 [00:00<?, ?it/s]

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/workspace/src/utils.py", line 101, in __getitem__
    dix = [self.stoi[s] for s in chunk[0]]
  File "/workspace/src/utils.py", line 101, in <listcomp>
    dix = [self.stoi[s] for s in chunk[0]]
KeyError: 4.0
